In [ ]:
import osmium
from tqdm import tqdm
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import transform
from shapely import intersects
import pyproj
from rtree import index
import matplotlib.pyplot as plt

import utils.projections as projections

In [ ]:
# Define a class to handle OSM data parsing
class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        super(OSMHandler, self).__init__()
        self.lakes = []
        self.bodies = ['lake', 'reservoir', 'pond']

    def way(self, w):
        if 'natural' in w.tags and w.tags['natural'] == 'water' and 'water' in w.tags and w.tags['water'] in self.bodies:
            lake_coords = []
            for n in w.nodes:
                if n.location.valid():
                    lake_coords.append(Point(n.location.lat, n.location.lon))
            if len(lake_coords) > 1:
                lake_meta = {
                    'id': w.id,
                    'name': w.tags['name'] if 'name' in w.tags else '',
                }
                line = LineString(lake_coords)
                if line.is_closed:
                    polygon = Polygon(lake_coords)
                    self.lakes.append((polygon, lake_meta))

# Load OSM data using OSMHandler
osm_handler = OSMHandler()
osm_handler.apply_file("data/extract.osm.pbf", locations=True, idx='flex_mem') # use idx=dense_mmap_array for large files on linux

In [ ]:
lakes = osm_handler.lakes

In [ ]:
BUFFER = 25
buffered_lakes = []
for (poly, meta) in tqdm(lakes):
    buffered_poly = projections.calculate_buffer(poly, BUFFER)
    buffered_lakes.append((buffered_poly, meta))

In [ ]:
# Create an R-tree index
lake_idx = index.Index()

# Insert polygons into the R-tree index with their bounding boxes
for i, (poly, _) in enumerate(buffered_lakes):
    bbox = poly.bounds
    lake_idx.insert(i, bbox)

# Function to find the polygon a line is in (O(log(N)) time complexity)
def find_lake_for_trail(line):
    for i in lake_idx.intersection(line.bounds):
        if intersects(buffered_lakes[i][0], line):
            return i  # Index of the polygon

In [ ]:
def map_trail_to_lake(trail_subset):
    first_encountered_lake_idx = []
    for (line, _) in tqdm(trail_subset):
        polygon_index = find_lake_for_trail(line)
        first_encountered_lake_idx.append(polygon_index)
    return first_encountered_lake_idx

In [ ]:
def print_trail_and_lake(trail, lake, buffered_lake):
    fig, ax = plt.subplots()

    rendered_trail = projections.project_line_to_meters(trail[0])
    rendered_lake = projections.project_poly_to_meters(lake[0])
    rendered_buffered_lake = projections.project_poly_to_meters(buffered_lake[0])

    # Plot polygons
    x, y = rendered_lake.exterior.xy
    ax.plot(x, y, label='Lake', color='r')
    x, y = rendered_buffered_lake.exterior.xy
    ax.plot(x, y, label='Buffered Lake', color='y')

    # Plot line
    x, y = rendered_trail.xy
    plt.plot(x, y, label='Trail', color='b')

    # Set equal aspect ratio
    plt.axis('equal')

    # Set labels and legend
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title("{} and {}".format(trail[1]['name'], lake[1]['name']))
    ax.legend()

    # Show the plot
    plt.show()

In [ ]:
import os
import pickle

filename = os.path.join("data", 'all_trail_segments.pickle')

with open(filename, 'rb') as f:
    hiking_trails = pickle.load(f)

trail_subset = hiking_trails

In [ ]:
lake_map = map_trail_to_lake(trail_subset)

In [ ]:
LIMIT = 15
count = 0
for i in range(len(trail_subset)):
    trail = trail_subset[i]

    lake_idx = lake_map[i]
    if lake_idx == None:
        continue

    lake = lakes[lake_idx]
    buffered_lake = buffered_lakes[lake_idx]
    print_trail_and_lake(trail, lake, buffered_lake)

    count += 1
    if count == LIMIT:
        break

In [ ]:
import os
import pickle

filename = os.path.join("data", 'all_lakes.pickle')
with open(filename, 'rb') as f:
    pickle.dump(lakes)

filename = os.path.join("data", 'trail_to_lake_map.pickle')

with open(filename, 'rb') as f:
    pickle.dump(lake_map)